In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
def power_law(x, n, a, b):
    return a*x**n+b

def curve_fit_power_law(xdata, ydata):
    popt, _ = curve_fit(power_law, xdata, ydata)
    return power_law(xdata,*popt), popt

In [3]:
foldername = "../SimResults/"
df = pd.read_csv("../SimResults/sim_1.10e+00r0_800L_31seed.csv")

In [4]:
df.head()

,t,num_active_sites,N,h_mean,h_std,trans_len,paral_len,density_last_interval,empty_bond_mass,empty_bond_volume,mean_bond_size
0,0.365739,0,1,0.00125,0.035333,1,1,0.0000,0.0,0.0,0.0
1,0.365739,0,2,0.00250,0.049937,1,1,0.0005,NaN,NaN,NaN


In [6]:
name_list = []
r0_list = [0.1, 0.5, 0.7, 0.9, 0.99, 0.999, 0.9999, 1, 1.001, 1.01, 1.1]
delta_list = [np.abs(np.exp(-r) - np.exp(-1)) for r in r0_list]
dom_list = [200, 400, 800]

max_survival_times_dom = []
transverse_length_dom = []
paral_length_dom = []

for dom in dom_list:
    max_survival_times_single_dom = np.zeros((len(r0_list), 2))
    transverse_length_single_dom = np.zeros((len(r0_list), 2))
    parallel_length_single_dom = np.zeros((len(r0_list), 2))

    for r, r_0 in enumerate(r0_list):
        exit_times = np.zeros(32)
        trans_len = np.zeros(32)
        paral_len = np.zeros(32)
        for i, seed in enumerate(range(32)):
            csv_name = '../SimResults/'+ 'sim_%.2er0_%dL_%dseed.csv'%(r_0, dom, seed)
            try:
                df = pd.read_csv(csv_name)
                exit_times[i] = df["t"].iloc[-1]
                trans_len[i] = df[" trans_len"].iloc[-1]
                paral_len[i] = df[" paral_len"].iloc[-1]
            except FileNotFoundError:
                exit_times[i] = np.NaN

            except KeyError:
                try:
                    df = pd.read_csv(csv_name)
                    exit_times[i] = df["t"].iloc[-1]
                    trans_len[i] = df["trans_len"].iloc[-1]
                    paral_len[i] = df["paral_len"].iloc[-1]
                except KeyError:
                    print("How did you fuck up this bad??")
        
        max_survival_times_single_dom[r, 0] = np.mean(exit_times)
        max_survival_times_single_dom[r, 1] = np.std(exit_times)
        transverse_length_single_dom[r, 0] = np.mean(trans_len)
        transverse_length_single_dom[r, 1] = np.std(trans_len)
        parallel_length_single_dom[r, 0] = np.mean(paral_len)
        parallel_length_single_dom[r, 1] = np.std(paral_len)
        
    max_survival_times_dom.append(max_survival_times_single_dom)
    transverse_length_dom.append(transverse_length_dom)
    paral_length_dom.append(paral_length_dom)

max_survival_times = np.array(max_survival_times_dom).squeeze()
transverse_length_dom = np.array(transverse_length_dom).squeeze()
paral_length = np.array(paral_length_dom).squeeze()

In [31]:
xdata = delta_list
ydata = max_survival_times_dom
ydata_std = max_survival_times_dom
yfit, popt = curve_fit_power_law(xdata, ydata)
plt.figure()
plt.plot(xdata, yfit, 'k')
plt.fill_between(xdata, ydata-ydata_std, ydata+ydata_std, color = 'b', alpha = 0.25)
plt.plot(xdata, ydata, 'b', label = r"$L = 200, T\sim\Delta^{-1.15}$")
plt.xlabel(r"$\Delta = |e^{-r_0\rho} - e^{-1}|$")
plt.ylabel("Survival Time")
plt.xscale("log")
plt.yscale("log")
plt.legend()

ValueError: operands could not be broadcast together with shapes (11,) (3,11,2) 

In [ ]:
name_list = []
r0_list = [0.1, 0.5, 0.7, 0.9, 0.99, 0.999, 0.9999, 1, 1.001, 1.01, 1.1]
delta_list = [np.abs(np.exp(-r) - np.exp(-1)) for r in r0_list]
dom_list = [200, 400, 800]

Masses_dom = []
Volumes_dom = []
Sizes_dom = []

for dom in dom_list:
    M_single_dom = np.zeros((len(r0_list), 2))
    V_single_dom = np.zeros((len(r0_list), 2))
    S_single_dom = np.zeros((len(r0_list), 2))

    for r, r_0 in enumerate(r0_list):
        M = np.zeros(32)
        V = np.zeros(32)
        S = np.zeros(32)
        for i, seed in enumerate(range(32)):
            csv_name = '../SimResults/'+ 'sim_%.2er0_%dL_%dseed.csv'%(r_0, dom, seed)
            try:
                df = pd.read_csv(csv_name)
                M[i] = df["t"].iloc[-1]
                V[i] = df["trans_len"].iloc[-1]
                S[i] = df["paral_len"].iloc[-1]
            except FileNotFoundError:
                M[i] = np.NaN
                V[i] = np.NaN
                S[i] = np.NaN
            except KeyError:
                try:
                    df = pd.read_csv(csv_name)
                    M[i] = df[" empty_bond_mass"].iloc[-1]
                    V[i] = df[" empty_bond_volume"].iloc[-1]
                    S[i] = df[" empty_bond_size"].iloc[-1]
                except KeyError:
                    print("How did you fuck this up?")
        
        M_single_dom[r, 0] = np.mean(exit_times)
        M_single_dom[r, 1] = np.std(exit_times)
        V_single_dom[r, 0] = np.mean(trans_len)
        V_single_dom[r, 1] = np.std(trans_len)
        S_single_dom[r, 0] = np.mean(paral_len)
        S_single_dom[r, 1] = np.std(paral_len)
        
    Masses_dom.append(M_single_dom)
    Volumes_dom.append(V_single_dom)
    Sizes_dom.append(S_single_dom)

Masses = np.array(Masses_dom).squeeze()
Volumes = np.array(Volumes_dom).squeeze()
Sizes = np.array(Sizes_dom).squeeze()
